In [1]:
import pandas as pd
import json,glob, os
import re
from datetime import datetime
import requests
import os


In [2]:
json_dir = 'merged_pr_15_02_2024.json'

In [3]:
pull_request_merged = []
f = open(json_dir)
data = json.load(f)
for i in data:
    pull_request_merged.append(i)
f.close()

In [4]:
len(pull_request_merged)

154

In [5]:
# We are interested in the merged PRs. Let create list of projects
# from the merged PR lists
items = []
for pr in pull_request_merged:
    items.append(pr.split('/pull/')[0])
projects = set(items)

In [6]:
len(projects)

133

In [7]:
# helper function
"""
    Read from list
"""
# token_file = 'tokens.txt'
token_file = '../../../tokens.txt'

token_list = []
with open(token_file, 'r') as f:
    for line in f.readlines():
        token_list.append(line.strip('\n'))

"""
    Manually add in notebook
"""
#token_list = ["nadHJok13mdkaamda", "ajd1819kdamda","..."]

lenTokens = len(token_list)

def get_response(url, token_list, ct):
    json_data = None

    # token_list, len_tokens = tokens()
    len_tokens = len(token_list)
    try:
        ct = ct % len_tokens
        headers = {'Authorization': 'Bearer {}'.format(token_list[ct])}
        request = requests.get(url, headers=headers)
        json_data = json.loads(request.content)
        ct += 1
    except Exception as e:
        print(e)
        print("Error in func: [get_response]...")
    return json_data, ct

In [8]:
def api_request(url, token):
    header = {'Authorization': 'token %s' % token}
    response = requests.get(url, headers=header)
    try:
        json_response = json.loads(response.content)
        return json_response
    except Exception as e:
        return response

In [9]:
#Filter `toy projects based on the following criteria
#---- 1. At least 2 developers ---
# 2. At least 100 commits
# 3. At least 1 review - this is pracific to pull request.
# two_developer = []
project_filter = []
for project in projects:
    # constract api url
    part = project.split('github.com/')
    # url = f'{part[0]}api.github.com/repos/{part[1]}/contributors'
    # try:
    #     request = api_request(url, token_list[0])
    #     # at least 2 developers
    #     if len(request) > 1:
    #         two_developer.append(project)

    # at least 100 commits
    try:
        commits_url = f'{part[0]}api.github.com/repos/{part[1]}/commits?per_page=100'
        fetch_commits = api_request(commits_url, token_list[0])
        if len(fetch_commits) >= 100:
            project_filter.append(project)
    except Exception as e:
        print("skipping...", e)
    # except Exception as e:
    #     print(project, e)

In [10]:
len(project_filter)

107

In [11]:
# get the merged PRs that are part of the remainings project and perform 
# PR specific filtering... i.e. PR should have at least 1 review
# e.g url: https://api.github.com/repos/apache/kafka/pulls/14540/comments

pull_request_clean = []
for pf in project_filter:
    for pr in pull_request_merged:
        pr_part = pr.split('/pull/')
        if pf == pr_part[0]:
            # at least 1 reviews
            pf_part = pf.split('github.com/')
            try:
                comments_url = f"{part[0]}api.github.com/repos/{pf_part[1]}/pulls/{pr_part[1]}/reviews"
                fetch_comments = api_request(comments_url, token_list[0])
                if len(fetch_comments) >= 1:
                    pull_request_clean.append(pr)
            except Exception as e:
                print("skipping...", e)

In [24]:
# total number of PRs that met the selection criteria 
# pull_request_clean = set(pull_request_clean)
pull_request_clean[0]

'https://github.com/Rhetos/Rhetos/pull/479'

In [31]:
json_di = 'pr_sharing_15_02_2024.json'

In [32]:
total_PR = []
f = open(json_di)
data = json.load(f)

In [33]:
len(data['Sources'])

215

In [36]:
for i in pull_request_clean:
    print(i)

https://github.com/Rhetos/Rhetos/pull/479
https://github.com/Regression-Games/RGUnityBots/pull/100
https://github.com/faker-js/faker/pull/2658
https://github.com/VOICEVOX/voicevox/pull/1790
https://github.com/VOICEVOX/voicevox/pull/1621
https://github.com/rapidsai/cuspatial/pull/1295
https://github.com/uchicago-cs/chigame/pull/265
https://github.com/localstack/localstack-blog.github.io/pull/10
https://github.com/jwjacobson/jazz_repertoire/pull/65
https://github.com/okp4/docs/pull/364
https://github.com/nishiki-tech/nishiki-frontend/pull/223
https://github.com/nishiki-tech/nishiki-frontend/pull/184
https://github.com/marqo-ai/marqo/pull/670
https://github.com/bitcoin/bitcoin/pull/28846
https://github.com/VyProductions/SeniorDesign/pull/57
https://github.com/VyProductions/SeniorDesign/pull/53
https://github.com/VyProductions/SeniorDesign/pull/56
https://github.com/VyProductions/SeniorDesign/pull/51
https://github.com/dragonflydb/dragonfly/pull/2153
https://github.com/PolicyEngine/policye